# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-841f86edd2-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Literary Movements and Divine Comedy explorative search") 

Consider the following exploratory information need:

> investigate the literary movements and the poets. In particular, compare some italian poets in terms of place of birth, number of literary works and education.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q40185`| The Divine Comedy  | node |





Also consider

```
wd:Q40185 ?p ?obj .
```

is the BGP to retrieve all **properties of The Divine Comedy**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:

1. Who wrote The Divine Comedy? (return the IRI and the label of the author)

2. Identify the BGP for poet

3. Identify the BGP for literary movement

4. Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

5. Identify the BGP for literary work

6. Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

7. Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

8. Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label). 

## Task 1
Who wrote The Divine Comedy? (return the IRI and the label of the author)

In [3]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q40185 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2551'), ('name', 'used metre')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de Fran

In [4]:
# Get informations about instance of property
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q40185 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q47461344'), ('name', 'written work')]
1


In [5]:
# Get informations about 'written work'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q47461344 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('name', 'said to be the same as')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
7


In [6]:
# Get informations superclass of 'written work'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q47461344 wdt:P279 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q17537576'), ('name', 'creative work')]
1


In [7]:
# Get informations about 'creative work'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q17537576 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('name', 'has quality')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P6366'), ('name', 'Microsoft Academic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P828'), ('name', 'has cause')]
[('p', 'http://www.w

In [8]:
# Get informations about qualities of 'creative work'
queryString = """
SELECT DISTINCT ?quality ?name
WHERE {
   wd:Q17537576 wdt:P1552 ?quality .
   ?quality sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('quality', 'http://www.wikidata.org/entity/Q170658'), ('name', 'creativity')]
[('quality', 'http://www.wikidata.org/entity/Q73568642'), ('name', 'copyright status of works')]
2


In [9]:
# Get informations about 'properties for this type' of 'creative work'
queryString = """
SELECT DISTINCT ?prop ?name
WHERE {
   wd:Q17537576 wdt:P1963 ?prop .
   ?prop sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/entity/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/entity/P921'), ('name', 'main subject')]
[('prop', 'http://www.wikidata.org/entity/P571'), ('name', 'inception')]
[('prop', 'http://www.wikidata.org/entity/P856'), ('name', 'official website')]
[('prop', 'http://www.wikidata.org/entity/P740'), ('name', 'location of formation')]
[('prop', 'http://www.wikidata.org/entity/P170'), ('name', 'creator')]
[('prop', 'http://www.wikidata.org/entity/P767'), ('name', 'contributor to the creative work or subject')]
[('prop', 'http://www.wikidata.org/entity/P136'), ('name', 'genre')]
[('prop', 'http://www.wikidata.org/entity/P1779'), ('name', 'possible creator')]
[('prop', 'http://www.wikidata.org/entity/P272'), ('name', 'production company')]
[('prop', 'http://www.wikidata.org/entity/P275'), ('name', 'copyright license')]
[('prop', 'http://www.wikidata.org/entity/P859'), ('name', 'sponsor')]
[('prop', 'http://www.wikidata.org/entity/P166')

In [10]:
# Get informations about instance of property
queryString = """
SELECT DISTINCT ?creator ?name
WHERE {
   wd:Q40185 wdt:P170 ?creator .
   ?creator sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [11]:
# Get informations about 'described by source' property
queryString = """
SELECT DISTINCT ?res ?name
WHERE {
   wd:Q40185 wdt:P1343 ?res .
   ?res sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('res', 'http://www.wikidata.org/entity/Q2041543'), ('name', 'Ottův slovník naučný')]
[('res', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia')]
2


In [12]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q40185 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2551'), ('name', 'used metre')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de Fran

In [13]:
# Get the author
queryString = """
SELECT DISTINCT ?author ?name
WHERE {
   wd:Q40185 wdt:P50 ?author .
   ?author sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
1


Final query for this task

In [14]:
queryString = """
SELECT DISTINCT ?author ?name
WHERE {
   wd:Q40185 wdt:P50 ?author .
   ?author sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
1


## Task 2
Identify the BGP for poet

In [15]:
# Get properties of 'Dante Alighieri'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1067 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7992'), ('name', "Poeti d'Italia in lingua latina author ID")]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'h

In [17]:
# 'Dante Alighieri' occupation
queryString = """
SELECT DISTINCT ?occupation ?name
WHERE {
   wd:Q1067 wdt:P106 ?occupation .
   ?occupation sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q12144794'), ('name', 'prosaist')]
[('occupation', 'http://www.wikidata.org/entity/Q15994177'), ('name', 'political theorist')]
[('occupation', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('occupation', 'http://www.wikidata.org/entity/Q482980'), ('name', 'author')]
[('occupation', 'http://www.wikidata.org/entity/Q4964182'), ('name', 'philosopher')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('occupation', 'http://www.wikidata.org/entity/Q822146'), ('name', 'lyricist')]
[('occupation', 'http://www.wikidata.org/entity/Q82955'), ('name', 'politician')]
[('occupation', 'http://www.wikidata.org/entity/Q58968'), ('name', 'intellectual')]
9


Final query for this task

In [3]:
# 'Dante Alighieri' occupation
queryString = """
SELECT DISTINCT ?occupation ?name
WHERE {
   wd:Q1067 wdt:P106 ?occupation .
   ?occupation sc:name ?name.
   FILTER REGEX(?name, "[Pp]oet")
}
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
1


## Task 3
Identify the BGP for literary movement

In [18]:
# 'Dante Alighieri' movement
queryString = """
SELECT DISTINCT ?mov ?name
WHERE {
   wd:Q1067 wdt:P135 ?mov .
   ?mov sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('mov', 'http://www.wikidata.org/entity/Q542036'), ('name', 'Dolce Stil Novo')]
1


In [19]:
# Get properties for 'Dolce Stil Novo'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q542036 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2924'), ('name', 'Great Russian Encyclopedia Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3365'), ('name', 'Treccani ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4223'), ('name', "Treccani's Enciclopedia Italiana ID")]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[(

Final query for this task

In [4]:
# Get class of 'Dolce Stil Novo'
queryString = """
SELECT DISTINCT ?class ?name
WHERE {
   wd:Q542036 wdt:P31 ?class .
   ?class sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('class', 'http://www.wikidata.org/entity/Q3326717'), ('name', 'literary movement')]
1


## Task 4
Return the poets born in the same place of the author of The Divine Comedy who are born between 1300 and 1500 (1300 and 1500 included) (the result set must be a list of couples poets IRI and label).

In [22]:
# Get all the properties of a Poet
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q49757 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org

In [26]:
# Get all subjects that have 'poet' as 'occupation' 
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1373516'), ('name', 'Eugénio de Castro')]
[('poet', 'http://www.wikidata.org/entity/Q589406'), ('name', 'Empress Xiaoquancheng')]
[('poet', 'http://www.wikidata.org/entity/Q610608'), ('name', 'António Aleixo')]
[('poet', 'http://www.wikidata.org/entity/Q710504'), ('name', 'Bohuslav Balbín')]
[('poet', 'http://www.wikidata.org/entity/Q8188119'), ('name', 'Adelina Lopes Vieira')]
[('poet', 'http://www.wikidata.org/entity/Q8201526'), ('name', 'Antonio Márquez Salas')]
[('poet', 'http://www.wikidata.org/entity/Q8201629'), ('name', 'Antonio Rendic')]
[('poet', 'http://www.wikidata.org/entity/Q8202094'), ('name', 'Antônio de Castro Assumpção')]
[('poet', 'http://www.wikidata.org/entity/Q8246828'), ('name', 'Bernat Català de Valleriola')]
[('poet', 'http://www.wikidata.org/entity/Q1191225'), ('name', 'Tan Sitong')]
10


In [26]:
# Get all subjects that have 'poet' as 'occupation' 
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1373516'), ('name', 'Eugénio de Castro')]
[('poet', 'http://www.wikidata.org/entity/Q589406'), ('name', 'Empress Xiaoquancheng')]
[('poet', 'http://www.wikidata.org/entity/Q610608'), ('name', 'António Aleixo')]
[('poet', 'http://www.wikidata.org/entity/Q710504'), ('name', 'Bohuslav Balbín')]
[('poet', 'http://www.wikidata.org/entity/Q8188119'), ('name', 'Adelina Lopes Vieira')]
[('poet', 'http://www.wikidata.org/entity/Q8201526'), ('name', 'Antonio Márquez Salas')]
[('poet', 'http://www.wikidata.org/entity/Q8201629'), ('name', 'Antonio Rendic')]
[('poet', 'http://www.wikidata.org/entity/Q8202094'), ('name', 'Antônio de Castro Assumpção')]
[('poet', 'http://www.wikidata.org/entity/Q8246828'), ('name', 'Bernat Català de Valleriola')]
[('poet', 'http://www.wikidata.org/entity/Q1191225'), ('name', 'Tan Sitong')]
10


In [27]:
# Get all the properties of 'Dante Alighieri'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1067 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7992'), ('name', "Poeti d'Italia in lingua latina author ID")]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'h

In [28]:
# Get 'place of birth' and 'date of birth' of 'Dante Alighieri'
queryString = """
SELECT DISTINCT ?dob ?place ?placeName
WHERE {
   wd:Q1067 wdt:P569 ?dob .
   wd:Q1067 wdt:P19 ?place .
   ?place sc:name ?placeName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('dob', '1265-01-01T00:00:00Z'), ('place', 'http://www.wikidata.org/entity/Q2044'), ('placeName', 'Florence')]
1


In [30]:
# Get all subjects that have 'poet' as 'occupation' that were born between 1300 and 1500 
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet sc:name ?name.
   
   ?poet wdt:P569 ?dob .
   FILTER (?dob >= "1300-01-01T00:00:00Z"^^xsd:dateTime && ?dob <= "1500-01-01T00:00:00Z"^^xsd:dateTime)
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q16075667'), ('name', 'Du Mu')]
[('poet', 'http://www.wikidata.org/entity/Q16039483'), ('name', 'Arias Barbosa')]
[('poet', 'http://www.wikidata.org/entity/Q562300'), ('name', 'Gilles Joye')]
[('poet', 'http://www.wikidata.org/entity/Q655618'), ('name', 'Francesco Berni')]
[('poet', 'http://www.wikidata.org/entity/Q296272'), ('name', 'Pietro Aretino')]
[('poet', 'http://www.wikidata.org/entity/Q5880389'), ('name', 'Girolamo Casio')]
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('name', 'Michel Verin')]
[('poet', 'http://www.wikidata.org/entity/Q22586308'), ('name', 'Domenico da Prato')]
[('poet', 'http://www.wikidata.org/entity/Q790956'), ('name', 'Avigdor Kara')]
[('poet', 'http://www.wikidata.org/entity/Q7141661'), ('name', 'Kakimon-in')]
10


In [33]:
# Get all subjects that have 'poet' as 'occupation' that were born between 1300 and 1500 and born in 'Florence'
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet sc:name ?name.
   
   ?poet wdt:P569 ?dob .
   FILTER (?dob >= "1300-01-01T00:00:00Z"^^xsd:dateTime && ?dob <= "1500-01-01T00:00:00Z"^^xsd:dateTime)
   
   ?poet wdt:P19 ?place .
   FILTER(?place = wd:Q2044)
}
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('name', 'Michel Verin')]
[('poet', 'http://www.wikidata.org/entity/Q1248954'), ('name', 'Giovanni di Bernardo Rucellai')]
[('poet', 'http://www.wikidata.org/entity/Q177854'), ('name', "Lorenzo de' Medici")]
[('poet', 'http://www.wikidata.org/entity/Q328117'), ('name', 'Girolamo Benivieni')]
[('poet', 'http://www.wikidata.org/entity/Q721846'), ('name', 'Luigi Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1399'), ('name', 'Niccolò Machiavelli')]
[('poet', 'http://www.wikidata.org/entity/Q366499'), ('name', 'Orcagna')]
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio')]
[('poet', 'http://www.wikidata.org/entity/Q631635'), ('name', 'Luigi Pulci')]
[('poet', 'http://www.wikidata.org/entity/Q2857170'), ('name', 'Antonio Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1527917'), ('name', 'Giuliano Dati')]
[('poet', 'http://www.wikidata.org/entity/Q45539848'), ('name', 'Niccolò Tinucc

Final query for this task

In [3]:
# Get all subjects that have 'poet' as 'occupation' that were born between 1300 (included) and 1500 (included) and born in 'Florence'
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet sc:name ?name.
   
   ?poet wdt:P569 ?dob .
   FILTER (?dob >= "1300-01-01T00:00:00Z"^^xsd:dateTime && ?dob <= "1500-12-31T23:59:59Z"^^xsd:dateTime)
   
   ?poet wdt:P19 ?place .
   FILTER(?place = wd:Q2044)
}
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q21003797'), ('name', 'Michel Verin')]
[('poet', 'http://www.wikidata.org/entity/Q1248954'), ('name', 'Giovanni di Bernardo Rucellai')]
[('poet', 'http://www.wikidata.org/entity/Q177854'), ('name', "Lorenzo de' Medici")]
[('poet', 'http://www.wikidata.org/entity/Q328117'), ('name', 'Girolamo Benivieni')]
[('poet', 'http://www.wikidata.org/entity/Q721846'), ('name', 'Luigi Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1399'), ('name', 'Niccolò Machiavelli')]
[('poet', 'http://www.wikidata.org/entity/Q366499'), ('name', 'Orcagna')]
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio')]
[('poet', 'http://www.wikidata.org/entity/Q631635'), ('name', 'Luigi Pulci')]
[('poet', 'http://www.wikidata.org/entity/Q2857170'), ('name', 'Antonio Alamanni')]
[('poet', 'http://www.wikidata.org/entity/Q1527917'), ('name', 'Giuliano Dati')]
[('poet', 'http://www.wikidata.org/entity/Q45539848'), ('name', 'Niccolò Tinucc

## Task 5
Identify the BGP for literary work

In [34]:
# Get informations about 'instance of' property of 'The Divine Commedy'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q40185 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q47461344'), ('name', 'written work')]
1


In [36]:
# Get properties of 'written work'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q47461344 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('name', 'uses')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('name', 'said to be the same as')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
7


In [37]:
# Get superclass of 'written work'
queryString = """
SELECT DISTINCT ?super ?name
WHERE {
   wd:Q47461344 wdt:P279 ?super .
   ?super sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('super', 'http://www.wikidata.org/entity/Q17537576'), ('name', 'creative work')]
1


In [38]:
# Get all instances of 'written work'
queryString = """
SELECT DISTINCT ?inst ?name
WHERE {
   ?inst wdt:P279 wd:Q47461344 .
   ?inst sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q1433985'), ('name', 'broadside')]
[('inst', 'http://www.wikidata.org/entity/Q58901591'), ('name', 'comparative study')]
[('inst', 'http://www.wikidata.org/entity/Q7254446'), ('name', 'pseudo-scholarship')]
[('inst', 'http://www.wikidata.org/entity/Q1501855'), ('name', 'party platform')]
[('inst', 'http://www.wikidata.org/entity/Q58898636'), ('name', 'evaluation study')]
[('inst', 'http://www.wikidata.org/entity/Q277759'), ('name', 'book series')]
[('inst', 'http://www.wikidata.org/entity/Q7397'), ('name', 'software')]
[('inst', 'http://www.wikidata.org/entity/Q7725634'), ('name', 'literary work')]
[('inst', 'http://www.wikidata.org/entity/Q13136'), ('name', 'reference work')]
[('inst', 'http://www.wikidata.org/entity/Q15087423'), ('name', 'ISO standard')]
[('inst', 'http://www.wikidata.org/entity/Q2352616'), ('name', 'catalogue')]
[('inst', 'http://www.wikidata.org/entity/Q5173771'), ('name', 'brochure')]
[('inst', 'http://www.wikidata

Final query for this task

In [5]:
queryString = """
SELECT DISTINCT ?inst ?name
WHERE {
   ?inst wdt:P279 wd:Q47461344 .
   ?inst sc:name ?name.
   FILTER REGEX (?name, "literary work")
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q7725634'), ('name', 'literary work')]
1


## Task 6
Compare the author of The Divine Comedy with Giovanni Boccaccio. Return for these poets the number of literary work they wrote (the result set must be two triples -one for each author- author IRI, label and #literary work).

In [47]:
# Get the properties of 'Human'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q5 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5806'), ('name', 'SNOMED CT identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1056'), ('name', 'product or material produced')]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P129'), ('name', 'physically interacts with')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Onlin

In [61]:
# Filter among 'Humans' people with name 'Giovanni ...' and surname 'Boccaccio'
queryString = """
SELECT DISTINCT ?human ?surname ?firstname
WHERE {
    ?human wdt:P31 wd:Q5. 
    ?human wdt:P734 ?familyName .
    ?familyName sc:name ?surname.
    
    FILTER REGEX(?surname,"^[Bb]occaccio")
    
    ?human wdt:P735 ?givenName .
    ?givenName sc:name ?firstname.
    FILTER REGEX(?firstname,"^[Gg]iovanni*")
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q3766842'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni')]
2


In [61]:
# Filter among 'Humans' people with name 'Giovanni ...' and surname 'Boccaccio'
queryString = """
SELECT DISTINCT ?human ?surname ?firstname
WHERE {
    ?human wdt:P31 wd:Q5. 
    ?human wdt:P734 ?familyName .
    ?familyName sc:name ?surname.
    
    FILTER REGEX(?surname,"^[Bb]occaccio")
    
    ?human wdt:P735 ?givenName .
    ?givenName sc:name ?firstname.
    FILTER REGEX(?firstname,"^[Gg]iovanni*")
    
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q3766842'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni')]
2


In [62]:
# Filter among 'Humans' people with name 'Giovanni ...' and surname 'Boccaccio' and get the occupation of matches
queryString = """
SELECT DISTINCT ?human ?surname ?firstname ?occupationName
WHERE {
    ?human wdt:P31 wd:Q5. 
    
    ?human wdt:P734 ?familyName .
    ?familyName sc:name ?surname.
    FILTER REGEX(?surname,"^[Bb]occaccio")
    
    ?human wdt:P735 ?givenName .
    ?givenName sc:name ?firstname.
    FILTER REGEX(?firstname,"^[Gg]iovanni*")
    
    ?human wdt:P106 ?occupation .
    ?occupation sc:name ?occupationName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q3766842'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'carabinier')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'short story writer')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'diplomat')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'translator')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'writer')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'poet')]
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni'), ('occupationName', 'biographer')]
[('human', 'http://w

In [63]:
# Filter among 'Humans' people with name 'Giovanni ...' and surname 'Boccaccio' and have occupation 'Poet'
queryString = """
SELECT DISTINCT ?human ?surname ?firstname
WHERE {
    ?human wdt:P31 wd:Q5. 
    
    ?human wdt:P734 ?familyName .
    ?familyName sc:name ?surname.
    FILTER REGEX(?surname,"^[Bb]occaccio")
    
    ?human wdt:P735 ?givenName .
    ?givenName sc:name ?firstname.
    FILTER REGEX(?firstname,"^[Gg]iovanni*")
    
    ?human wdt:P106 ?occupation .
    FILTER(?occupation = wd:Q49757)
}
"""

print("Results")
x=run_query(queryString)

Results
[('human', 'http://www.wikidata.org/entity/Q1402'), ('surname', 'Boccaccio'), ('firstname', 'Giovanni')]
1


In [64]:
# get all properties of 'Giovanni Boccaccio'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q1402 ?p ?obj .
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2018'), ('name', 'Teuchos ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7992'), ('name', "Poeti d'Italia in lingua latina author ID")]
[('p', 'http://www.wikidata.org/prop/direct/P8975'), ('name', 'Manus Online author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P1150'), ('name', 'Regensburg Classification')]
[('p', 'http://ww

In [4]:
# get all properties of 'Giovanni Boccaccio'
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   wd:Q1402 ?p ?o .
   ?p sc:name ?pname.
   ?o sc:name ?oname.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q15949613'), ('oname', 'short story writer')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pname', 'place of death'), ('o', 'http://www.wikidata.org/entity/Q82646'), ('oname', 'Certaldo')]
[('p', 'http://www.wikidata.org/prop/direct/P800'), ('pname', 'notable work'), ('o', 'http://www.wikidata.org/entity/Q16438'), ('oname', 'The Decameron')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('pname', 'country of citizenship'), ('o', 'http://www.wikidata.org/entity/Q148540'), ('oname', 'Republic of Florence')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q193391'), ('oname', 'diplomat')]
[(

In [65]:
# Get all 'written work' that have as author 'Giovanni Boccaccio'
queryString = """
SELECT DISTINCT ?ww ?name
WHERE {
   ?ww wdt:P31 wd:Q47461344 .
   ?ww wdt:P50 wd:Q1402 .
   ?ww sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('ww', 'http://www.wikidata.org/entity/Q3759459'), ('name', 'Genealogia deorum gentilium')]
[('ww', 'http://www.wikidata.org/entity/Q20630337'), ('name', 'De montibus')]
[('ww', 'http://www.wikidata.org/entity/Q3792638'), ('name', 'Corbaccio')]
[('ww', 'http://www.wikidata.org/entity/Q42192295'), ('name', 'The Life of Dante')]
4


In [75]:
# Get all 'literary work' that have as author 'Giovanni Boccaccio'
queryString = """
SELECT DISTINCT ?lw ?name
WHERE {
   ?lw wdt:P31 wd:Q7725634 .
   ?lw wdt:P50 wd:Q1402 .
   ?lw sc:name ?name.
}
ORDER BY ASC(?lw)
"""

print("Results")
x=run_query(queryString)

Results
[('lw', 'http://www.wikidata.org/entity/Q16438'), ('name', 'The Decameron')]
[('lw', 'http://www.wikidata.org/entity/Q3614628'), ('name', 'Amorosa visione')]
[('lw', 'http://www.wikidata.org/entity/Q3684237'), ('name', 'Comedia delle ninfe fiorentine')]
[('lw', 'http://www.wikidata.org/entity/Q3721314'), ('name', 'Elegia di Madonna Fiammetta')]
[('lw', 'http://www.wikidata.org/entity/Q3745486'), ('name', 'The Filocolo')]
[('lw', 'http://www.wikidata.org/entity/Q3877086'), ('name', 'Ninfale fiesolano')]
[('lw', 'http://www.wikidata.org/entity/Q5244311'), ('name', 'De casibus virorum illustrium')]
[('lw', 'http://www.wikidata.org/entity/Q71736142'), ('name', 'De Canaria')]
[('lw', 'http://www.wikidata.org/entity/Q86673964'), ('name', 'Mavortis Milex')]
9


In [71]:
# Get number of 'literary work' that have as author 'Giovanni Boccaccio'
queryString = """
SELECT ?author ?authorName (COUNT(DISTINCT ?lw) as ?literaryWorks)
WHERE {
   ?lw wdt:P31 wd:Q7725634 .
   ?lw wdt:P50 ?author .
   ?author sc:name ?authorName.
   FILTER(?author = wd:Q1402)
}
GROUP BY ?author ?authorName
"""

print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1402'), ('authorName', 'Giovanni Boccaccio'), ('literaryWorks', '10')]
1


In [76]:
# Get 'literary work' that have as author 'Giovanni Boccaccio'
queryString = """
SELECT ?author ?lw
WHERE {
   ?lw wdt:P31 wd:Q7725634 .
   ?lw wdt:P50 ?author .
   ?author sc:name ?authorName.
   FILTER(?author = wd:Q1402)
}
ORDER BY ASC(?lw)
"""

print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q16438')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q3614628')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q3649284')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q3684237')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q3721314')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q3745486')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q3877086')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q5244311')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw', 'http://www.wikidata.org/entity/Q71736142')]
[('author', 'http://www.wikidata.org/entity/Q1402'), ('lw

Final query for this task

In [77]:
# Comparing literary works of 'Giovanni Boccaccio' with 'Dante Alighieri'
queryString = """
SELECT ?author ?authorName (COUNT(DISTINCT ?lw) as ?literaryWorks)
WHERE {
   ?lw wdt:P31 wd:Q7725634 .
   ?lw wdt:P50 ?author .
   ?author sc:name ?authorName.
   FILTER(?author = wd:Q1402 || ?author = wd:Q1067)
}
GROUP BY ?author ?authorName
"""

print("Results")
x=run_query(queryString)

Results
[('author', 'http://www.wikidata.org/entity/Q1402'), ('authorName', 'Giovanni Boccaccio'), ('literaryWorks', '10')]
[('author', 'http://www.wikidata.org/entity/Q1067'), ('authorName', 'Dante Alighieri'), ('literaryWorks', '37')]
2


## Task 7
Consider italian poets, their place of birth (the city) and their university education. Return for each city the number of poets who born there and who was educated there (the result set must be a list of triples city IRI, label and #poets).

In [78]:
# Get all subjects that have 'poet' as 'occupation'
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1373516'), ('name', 'Eugénio de Castro')]
[('poet', 'http://www.wikidata.org/entity/Q589406'), ('name', 'Empress Xiaoquancheng')]
[('poet', 'http://www.wikidata.org/entity/Q610608'), ('name', 'António Aleixo')]
[('poet', 'http://www.wikidata.org/entity/Q710504'), ('name', 'Bohuslav Balbín')]
[('poet', 'http://www.wikidata.org/entity/Q8188119'), ('name', 'Adelina Lopes Vieira')]
[('poet', 'http://www.wikidata.org/entity/Q8201526'), ('name', 'Antonio Márquez Salas')]
[('poet', 'http://www.wikidata.org/entity/Q8201629'), ('name', 'Antonio Rendic')]
[('poet', 'http://www.wikidata.org/entity/Q8202094'), ('name', 'Antônio de Castro Assumpção')]
[('poet', 'http://www.wikidata.org/entity/Q8246828'), ('name', 'Bernat Català de Valleriola')]
[('poet', 'http://www.wikidata.org/entity/Q1191225'), ('name', 'Tan Sitong')]
10


In [80]:
# Get 'ethnic group' of 'Dante Alighieri'
queryString = """
SELECT DISTINCT ?e ?name
WHERE {
   wd:Q1067 wdt:P172 ?e .
   ?e sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('e', 'http://www.wikidata.org/entity/Q50001'), ('name', 'Italians')]
1


I assume italian poets are all poets with `ethnic group` wd:Q50001 (Italians)

In [81]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group'
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet wdt:P172 wd:Q50001 .
   ?poet sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q296272'), ('name', 'Pietro Aretino')]
[('poet', 'http://www.wikidata.org/entity/Q982512'), ('name', 'Giuseppe Giusti')]
[('poet', 'http://www.wikidata.org/entity/Q55433'), ('name', 'Michelangelo Antonioni')]
[('poet', 'http://www.wikidata.org/entity/Q83321'), ('name', 'Giacomo Casanova')]
[('poet', 'http://www.wikidata.org/entity/Q182046'), ('name', 'Leon Battista Alberti')]
[('poet', 'http://www.wikidata.org/entity/Q48900'), ('name', 'Ludovico Ariosto')]
[('poet', 'http://www.wikidata.org/entity/Q165534'), ('name', 'Carlo Goldoni')]
[('poet', 'http://www.wikidata.org/entity/Q7803'), ('name', 'Bronzino')]
[('poet', 'http://www.wikidata.org/entity/Q982387'), ('name', 'Giovanni Capurro')]
[('poet', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
10


In [82]:
# Get 'place of birth' of 'Dante Alighieri'
queryString = """
SELECT DISTINCT ?pob ?pobName
WHERE {
   wd:Q1067 wdt:P19 ?pob .
   ?pob sc:name ?pobName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('pob', 'http://www.wikidata.org/entity/Q2044'), ('pobName', 'Florence')]
1


In [83]:
# Get all the cities in which italian poets were born
queryString = """
SELECT DISTINCT ?pob ?pobName
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet wdt:P172 wd:Q50001 .
   ?poet wdt:P19 ?pob .
   ?pob sc:name ?pobName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('pob', 'http://www.wikidata.org/entity/Q13378'), ('pobName', 'Arezzo')]
[('pob', 'http://www.wikidata.org/entity/Q1449'), ('pobName', 'Genoa')]
[('pob', 'http://www.wikidata.org/entity/Q1722'), ('pobName', 'Dubrovnik')]
[('pob', 'http://www.wikidata.org/entity/Q219330'), ('pobName', 'Pietrasanta')]
[('pob', 'http://www.wikidata.org/entity/Q52069'), ('pobName', 'Caprese Michelangelo')]
[('pob', 'http://www.wikidata.org/entity/Q1335'), ('pobName', 'Montevideo')]
[('pob', 'http://www.wikidata.org/entity/Q13362'), ('pobName', 'Ferrara')]
[('pob', 'http://www.wikidata.org/entity/Q144880'), ('pobName', 'Zakynthos')]
[('pob', 'http://www.wikidata.org/entity/Q2044'), ('pobName', 'Florence')]
[('pob', 'http://www.wikidata.org/entity/Q220'), ('pobName', 'Rome')]
[('pob', 'http://www.wikidata.org/entity/Q2634'), ('pobName', 'Naples')]
[('pob', 'http://www.wikidata.org/entity/Q2683'), ('pobName', 'Parma')]
[('pob', 'http://www.wikidata.org/entity/Q490'), ('pobName', 'Milan')]
[('pob', 'h

In [85]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
queryString = """
SELECT DISTINCT ?poet ?name ?dob
WHERE {
   ?poet wdt:P106 wd:Q49757 .
   ?poet wdt:P172 wd:Q50001 .
   ?poet wdt:P569 ?dob .
   ?poet sc:name ?name.
}
ORDER BY DESC(?dob)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q53009'), ('name', 'Bernardo Bertolucci'), ('dob', '1941-03-16T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q765'), ('name', 'Dario Fo'), ('dob', '1926-03-24T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q55433'), ('name', 'Michelangelo Antonioni'), ('dob', '1912-09-29T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q560541'), ('name', 'Tommaso Landolfi'), ('dob', '1908-08-09T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q334147'), ('name', 'Julius Evola'), ('dob', '1898-05-19T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q7368989'), ('name', 'Rosita Melo'), ('dob', '1897-07-09T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q311802'), ('name', 'Giuseppe Ungaretti'), ('dob', '1888-02-10T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q311802'), ('name', 'Giuseppe Ungaretti'), ('dob', '1888-02-08T00:00:00Z')]
[('poet', 'http://www.wikidata.org/entity/Q133855'), ('name', 'Guillaume

In [86]:
# Get all properties of 'Bernardo Bertolucci'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q53009 ?p ?o.
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('name', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('name', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID

In [88]:
# Get info about 'Bernardo Bertolucci' education
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q53009 wdt:P69 ?o.
   ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q209344'), ('name', 'Sapienza University of Rome')]
1


In [90]:
# Get info about 'Sapienza University of Rome'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q209344 ?p ?o.
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P9871'), ('name', 'INAPP author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P1128'), ('name', 'employees')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1416'), ('name', 'affiliation')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', '

In [93]:
# Get info about 'Sapienza University of Rome'
queryString = """
SELECT DISTINCT ?instanceOf ?name
WHERE {
   wd:Q209344 wdt:P31 ?instanceOf.
   ?instanceOf sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('instanceOf', 'http://www.wikidata.org/entity/Q875538'), ('name', 'public university')]
1


In [94]:
# Get info about 'public university'
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q875538 ?p ?o.
   ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('name', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P349'), ('name', 'National Diet Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons ca

In [95]:
# Get info about super class of 'public university'
queryString = """
SELECT DISTINCT ?super ?name
WHERE {
   wd:Q875538 wdt:P279 ?super.
   ?super sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('super', 'http://www.wikidata.org/entity/Q23002037'), ('name', 'public educational institution')]
[('super', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
2


In [109]:
# Get info country of 'Sapienza University of Rome'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q209344 wdt:P17 ?o.
   ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q38'), ('name', 'Italy')]
1


In [91]:
# Get info location of 'Sapienza University of Rome'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q209344 wdt:P131 ?o.
   ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q220'), ('name', 'Rome')]
1


In [115]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about theirs studies
queryString = """
SELECT ?educatedAt ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?educatedAt .    #BGP for educated at
   
   ?educatedAt sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('educatedAt', 'http://www.wikidata.org/entity/Q209344'), ('name', 'Sapienza University of Rome')]
[('educatedAt', 'http://www.wikidata.org/entity/Q209344'), ('name', 'Sapienza University of Rome')]
[('educatedAt', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('educatedAt', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('educatedAt', 'http://www.wikidata.org/entity/Q680971'), ('name', 'Experimental Centre of Cinematography')]
[('educatedAt', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('educatedAt', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('educatedAt', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('educatedAt', 'http://www.wikidata.org/entity/Q152171'), ('name', 'University of Bonn')]
[('educatedAt', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('educatedAt

In [114]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get the universities 
queryString = """
SELECT ?uni ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?educatedAt .    #BGP for educated at
   ?educatedAt wdt:P31 ?uni.
   
   ?uni sc:name ?name

}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q3953379'), ('name', 'superior graduate school in Italy')]
[('uni', 'http://www.wikidata.org/entity/Q2418495'), ('name', 'independent school')]
[('uni', 'http://www.wikidata.org/entity/Q1966910'), ('name', 'national academy')]
[('uni', 'http://www.wikidata.org/entity/Q1352795'), ('name', 'cinematheque')]
[('uni', 'http://www.wikidata.org/entity/Q380093'), ('name', 'film school')]
[('uni', 'http://www.wikidata.org/entity/Q207694'), ('name', 'art museum')]
[('uni', 'http://www.wikidata.org/entity/Q2668072'), ('name', 'collection')]
[('uni', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('uni', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('uni', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('uni', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('uni', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
[('uni', 'http://www.wikidata.org/entity/Q391

In [126]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about their studies
queryString = """
SELECT ?educatedAt ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?educatedAt .    #BGP for educated at
   ?educatedAt wdt:P31 wd:Q3918.  #BGP for universities
   
   ?educatedAt sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('educatedAt', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('educatedAt', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('educatedAt', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('educatedAt', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('educatedAt', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('educatedAt', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('educatedAt', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('educatedAt', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('educatedAt', 'http://www.wikidata.org/entity/Q955609'), ('name', 'University of Ferrara')]
[('educatedAt', 'http://www.wikidata.org/entity/Q593321'), ('name', 'University of Genoa')]
[('educatedAt', 'http://www.wikidata.org/ent

In [127]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about their university
queryString = """
SELECT DISTINCT ?educatedAt ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?educatedAt .    #BGP for educated at
   ?educatedAt wdt:P31 wd:Q3918.  #BGP for universities
   
   ?educatedAt sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('educatedAt', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('educatedAt', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('educatedAt', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('educatedAt', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('educatedAt', 'http://www.wikidata.org/entity/Q955609'), ('name', 'University of Ferrara')]
[('educatedAt', 'http://www.wikidata.org/entity/Q593321'), ('name', 'University of Genoa')]
[('educatedAt', 'http://www.wikidata.org/entity/Q645663'), ('name', 'University of Pisa')]
7


In [128]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about the university in which they conduct their university studies
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?uni .    #BGP for educated at
   ?uni wdt:P31 wd:Q3918.  #BGP for universities
   ?uni wdt:P17 wd:Q38.    #BGP to check that the univeristy is in Italy
   
   ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('uni', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('uni', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('uni', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('uni', 'http://www.wikidata.org/entity/Q955609'), ('name', 'University of Ferrara')]
[('uni', 'http://www.wikidata.org/entity/Q593321'), ('name', 'University of Genoa')]
[('uni', 'http://www.wikidata.org/entity/Q645663'), ('name', 'University of Pisa')]
7


In [129]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about the city in which they conduct their university studies
queryString = """
SELECT DISTINCT ?city ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?uni .             #BGP for educated at
   ?uni wdt:P31 wd:Q3918.           #BGP for universities
   ?uni wdt:P17 wd:Q38.             #BGP to check that the univeristy is in Italy
   ?uni wdt:P131 ?city.             #BGP to retieve the city 
   
   ?city sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q1449'), ('name', 'Genoa')]
[('city', 'http://www.wikidata.org/entity/Q13362'), ('name', 'Ferrara')]
[('city', 'http://www.wikidata.org/entity/Q13375'), ('name', 'Pisa')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('name', 'Florence')]
[('city', 'http://www.wikidata.org/entity/Q6259'), ('name', 'Pavia')]
[('city', 'http://www.wikidata.org/entity/Q279'), ('name', 'Modena')]
[('city', 'http://www.wikidata.org/entity/Q617'), ('name', 'Padua')]
7


In [131]:
# Get info about public university
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    wd:Q875538 ?p ?o .
    ?p sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1149'), ('name', 'Library of Congress Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P2163'), ('name', 'FAST ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P349'), ('name', 'National Diet Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons ca

In [133]:
# Get info about public university
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    wd:Q875538 wdt:P279 ?o .
    ?o sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q23002037'), ('name', 'public educational institution')]
[('o', 'http://www.wikidata.org/entity/Q3918'), ('name', 'university')]
2


In [7]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about the city in which they conduct their university studies
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .     #BGP for poet
   ?poet wdt:P172 wd:Q50001 .     #BGP for 'Italians'
   
   ?poet wdt:P69 ?uni .                  #BGP for educated at
   
   ?uni (wdt:P31)|(wdt:P279+) wd:Q3918.  #BGP for universities (missing 'Sapienza' public university)
   
   ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('uni', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('uni', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('uni', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('uni', 'http://www.wikidata.org/entity/Q955609'), ('name', 'University of Ferrara')]
[('uni', 'http://www.wikidata.org/entity/Q593321'), ('name', 'University of Genoa')]
[('uni', 'http://www.wikidata.org/entity/Q645663'), ('name', 'University of Pisa')]
7


In [8]:
# Get all universities in Italy
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
    ?uni wdt:P31 wd:Q3918. #BGP for instance of universtity
    ?uni wdt:P17 wd:Q38.   #BGP for country 'Italy'
    ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q7209754'), ('name', 'Polimoda')]
[('uni', 'http://www.wikidata.org/entity/Q1752540'), ('name', 'University of Calabria')]
[('uni', 'http://www.wikidata.org/entity/Q3551649'), ('name', 'University of Campania Studies "Luigi Vanvitelli"')]
[('uni', 'http://www.wikidata.org/entity/Q1546906'), ('name', "Ca' Foscari University of Venice")]
[('uni', 'http://www.wikidata.org/entity/Q727492'), ('name', 'IUAV University of Venice')]
[('uni', 'http://www.wikidata.org/entity/Q600924'), ('name', 'Foreigners University of Siena')]
[('uni', 'http://www.wikidata.org/entity/Q7198162'), ('name', 'Pisa University System')]
[('uni', 'http://www.wikidata.org/entity/Q1780458'), ('name', 'Tuscia University')]
[('uni', 'http://www.wikidata.org/entity/Q1440288'), ('name', 'Università Campus Bio-Medico')]
[('uni', 'http://www.wikidata.org/entity/Q1245999'), ('name', 'University of Siena')]
[('uni', 'http://www.wikidata.org/entity/Q2297486'), ('name', 'Universit

In [12]:
# Get universities and public universities in Italy
queryString = """
SELECT DISTINCT ?uni ?name ?class
WHERE {
    ?uni wdt:P17 wd:Q38.         #BGP for country 'Italy'
    ?uni wdt:P31 ?of.            #BGP for instance of
    FILTER(?of = wd:Q3918 || ?of = wd:Q875538) #filter universities and public universities
    ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q7209754'), ('name', 'Polimoda')]
[('uni', 'http://www.wikidata.org/entity/Q1752540'), ('name', 'University of Calabria')]
[('uni', 'http://www.wikidata.org/entity/Q3551649'), ('name', 'University of Campania Studies "Luigi Vanvitelli"')]
[('uni', 'http://www.wikidata.org/entity/Q1546906'), ('name', "Ca' Foscari University of Venice")]
[('uni', 'http://www.wikidata.org/entity/Q727492'), ('name', 'IUAV University of Venice')]
[('uni', 'http://www.wikidata.org/entity/Q600924'), ('name', 'Foreigners University of Siena')]
[('uni', 'http://www.wikidata.org/entity/Q7198162'), ('name', 'Pisa University System')]
[('uni', 'http://www.wikidata.org/entity/Q1780458'), ('name', 'Tuscia University')]
[('uni', 'http://www.wikidata.org/entity/Q1440288'), ('name', 'Università Campus Bio-Medico')]
[('uni', 'http://www.wikidata.org/entity/Q1245999'), ('name', 'University of Siena')]
[('uni', 'http://www.wikidata.org/entity/Q2297486'), ('name', 'Universit

The university of Rome 'La Sapienza' not an instance of 'University' but is an instance of a sub class of 'University', so with this query I want to retrieve all instances (or instances of a sub class) of university

In [16]:
# Get all universities in Italy
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
    ?uni wdt:P17 wd:Q38.             #BGP for country 'Italy'
    ?uni wdt:P31/wdt:P279* wd:Q3918. #BGP for instance of university or a subclass
    ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q7209754'), ('name', 'Polimoda')]
[('uni', 'http://www.wikidata.org/entity/Q1752540'), ('name', 'University of Calabria')]
[('uni', 'http://www.wikidata.org/entity/Q3551649'), ('name', 'University of Campania Studies "Luigi Vanvitelli"')]
[('uni', 'http://www.wikidata.org/entity/Q1546906'), ('name', "Ca' Foscari University of Venice")]
[('uni', 'http://www.wikidata.org/entity/Q548435'), ('name', 'Pontifical University of the Holy Cross')]
[('uni', 'http://www.wikidata.org/entity/Q498107'), ('name', 'Pontifical Institute of Arab and Islamic Studies')]
[('uni', 'http://www.wikidata.org/entity/Q727492'), ('name', 'IUAV University of Venice')]
[('uni', 'http://www.wikidata.org/entity/Q7228275'), ('name', 'Pontifical Roman Athenaeum S. Apollinare')]
[('uni', 'http://www.wikidata.org/entity/Q600924'), ('name', 'Foreigners University of Siena')]
[('uni', 'http://www.wikidata.org/entity/Q7198162'), ('name', 'Pisa University System')]
[('uni', 'h

In [17]:
# Get all cities in which there is at least university in Italy
queryString = """
SELECT DISTINCT ?city ?name
WHERE {
    ?uni wdt:P17 wd:Q38.             #BGP for country 'Italy'
    ?uni wdt:P31/wdt:P279* wd:Q3918. #BGP for instance of university or a subclass
    ?uni wdt:P131 ?city.             #BGP to retieve the city 
   
    ?city sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q13619'), ('name', 'Cosenza')]
[('city', 'http://www.wikidata.org/entity/Q6526'), ('name', 'Bolzano')]
[('city', 'http://www.wikidata.org/entity/Q13395'), ('name', 'Viterbo')]
[('city', 'http://www.wikidata.org/entity/Q2759'), ('name', 'Urbino')]
[('city', 'http://www.wikidata.org/entity/Q13666'), ('name', 'Messina')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('name', 'Florence')]
[('city', 'http://www.wikidata.org/entity/Q220'), ('name', 'Rome')]
[('city', 'http://www.wikidata.org/entity/Q2751'), ('name', 'Siena')]
[('city', 'http://www.wikidata.org/entity/Q3437'), ('name', 'Perugia')]
[('city', 'http://www.wikidata.org/entity/Q3519'), ('name', 'Bari')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('name', 'Milan')]
[('city', 'http://www.wikidata.org/entity/Q495'), ('name', 'Turin')]
[('city', 'http://www.wikidata.org/entity/Q6259'), ('name', 'Pavia')]
[('city', 'http://www.wikidata.org/entity/Q641'), ('name', 'Venice')]


In [18]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about the city in which they conduct their university studies
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .        #BGP for poet
   ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
   ?poet wdt:P69 ?uni .              #BGP for educated at
   ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
   
   ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q209344'), ('name', 'Sapienza University of Rome')]
[('uni', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('uni', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('uni', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('uni', 'http://www.wikidata.org/entity/Q152171'), ('name', 'University of Bonn')]
[('uni', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('uni', 'http://www.wikidata.org/entity/Q209842'), ('name', 'University of Paris')]
[('uni', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('uni', 'http://www.wikidata.org/entity/Q955609'), ('name', 'University of Ferrara')]
[('uni', 'http://www.wikidata.org/entity/Q593321'), ('name', 'University of Genoa')]
[('uni', 'http://www.wikidata.org/entity/Q645663'), ('name', 'University of Pisa')]
[('uni', 'http://www.wik

In [19]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about the city in which they conduct their university studies
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .        #BGP for poet
   ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
   ?poet wdt:P69 ?uni .              #BGP for educated at
   ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
   ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
   ?uni sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q209344'), ('name', 'Sapienza University of Rome')]
[('uni', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'University of Modena and Reggio Emilia')]
[('uni', 'http://www.wikidata.org/entity/Q219317'), ('name', 'University of Pavia')]
[('uni', 'http://www.wikidata.org/entity/Q131262'), ('name', 'University of Bologna')]
[('uni', 'http://www.wikidata.org/entity/Q193510'), ('name', 'University of Padua')]
[('uni', 'http://www.wikidata.org/entity/Q820887'), ('name', 'University of Florence')]
[('uni', 'http://www.wikidata.org/entity/Q955609'), ('name', 'University of Ferrara')]
[('uni', 'http://www.wikidata.org/entity/Q593321'), ('name', 'University of Genoa')]
[('uni', 'http://www.wikidata.org/entity/Q645663'), ('name', 'University of Pisa')]
9


In [20]:
# Get all subjects that have 'poet' as 'occupation' and 'Italians' as 'ethnic group' and their date of birth
# Get informations about the city in which they conduct their university studies
queryString = """
SELECT DISTINCT ?uni ?name
WHERE {
   ?poet wdt:P106 wd:Q49757 .        #BGP for poet
   ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
   ?poet wdt:P69 ?uni .              #BGP for educated at
   ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
   ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
   ?uni wdt:P131 ?city.              #BGP to retieve the city 
   ?city sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('uni', 'http://www.wikidata.org/entity/Q593321'), ('name', 'Genoa')]
[('uni', 'http://www.wikidata.org/entity/Q955609'), ('name', 'Ferrara')]
[('uni', 'http://www.wikidata.org/entity/Q645663'), ('name', 'Pisa')]
[('uni', 'http://www.wikidata.org/entity/Q131262'), ('name', 'Bologna')]
[('uni', 'http://www.wikidata.org/entity/Q820887'), ('name', 'Florence')]
[('uni', 'http://www.wikidata.org/entity/Q209344'), ('name', 'Rome')]
[('uni', 'http://www.wikidata.org/entity/Q219317'), ('name', 'Pavia')]
[('uni', 'http://www.wikidata.org/entity/Q1243705'), ('name', 'Modena')]
[('uni', 'http://www.wikidata.org/entity/Q193510'), ('name', 'Padua')]
9


In [23]:
# Get all the cities in which italian poets were born or attended university
queryString = """
SELECT DISTINCT ?city ?cityName
WHERE {

    {
        # Get all the cities in which Italian poets were born
        SELECT ?city ?cityName
        WHERE {
           ?poet wdt:P106 wd:Q49757 .
           ?poet wdt:P172 wd:Q50001 .
           ?poet wdt:P19 ?city .
           ?city sc:name ?cityName.
        }
    } UNION {
        # Get all the cities in which Italian poets attended university
        SELECT ?city ?cityName
        WHERE {
           ?poet wdt:P106 wd:Q49757 .        #BGP for poet
           ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
           ?poet wdt:P69 ?uni .              #BGP for educated at
           ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
           ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
           ?uni wdt:P131 ?city.              #BGP to retieve the city 
           ?city sc:name ?cityName.
        }
    }

}

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q13378'), ('cityName', 'Arezzo')]
[('city', 'http://www.wikidata.org/entity/Q1449'), ('cityName', 'Genoa')]
[('city', 'http://www.wikidata.org/entity/Q1722'), ('cityName', 'Dubrovnik')]
[('city', 'http://www.wikidata.org/entity/Q219330'), ('cityName', 'Pietrasanta')]
[('city', 'http://www.wikidata.org/entity/Q52069'), ('cityName', 'Caprese Michelangelo')]
[('city', 'http://www.wikidata.org/entity/Q1335'), ('cityName', 'Montevideo')]
[('city', 'http://www.wikidata.org/entity/Q13362'), ('cityName', 'Ferrara')]
[('city', 'http://www.wikidata.org/entity/Q144880'), ('cityName', 'Zakynthos')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('cityName', 'Florence')]
[('city', 'http://www.wikidata.org/entity/Q220'), ('cityName', 'Rome')]
[('city', 'http://www.wikidata.org/entity/Q2634'), ('cityName', 'Naples')]
[('city', 'http://www.wikidata.org/entity/Q2683'), ('cityName', 'Parma')]
[('city', 'http://www.wikidata.org/entity/Q490'), ('cityNa

In [25]:
# Given a city retrieve the Italian poets that were born there
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {

    ?poet wdt:P106 wd:Q49757 . #BGP for poet
    ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
    ?poet sc:name ?name.

    ?poet wdt:P19 ?city .
    FILTER(?city = wd:Q220)    #Filter only for city 'Rome'
}

"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q29473'), ('name', 'Pietro Metastasio')]
[('poet', 'http://www.wikidata.org/entity/Q133855'), ('name', 'Guillaume Apollinaire')]
[('poet', 'http://www.wikidata.org/entity/Q334147'), ('name', 'Julius Evola')]
3


In [26]:
# Given a city retrieve the Italian poets that were born there
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {

    ?poet wdt:P106 wd:Q49757 . #BGP for poet
    ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
    ?poet sc:name ?name.

    ?poet wdt:P19 ?city .
    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}

"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q7803'), ('name', 'Bronzino')]
[('poet', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
[('poet', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio')]
[('poet', 'http://www.wikidata.org/entity/Q2252971'), ('name', 'Vincenzo da Filicaja')]
4


In [29]:
# Given a city retrieve the number of Italian poets that were born there
queryString = """
SELECT COUNT(DISTINCT (?poet)) as ?born
WHERE {

    ?poet wdt:P106 wd:Q49757 . #BGP for poet
    ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
    ?poet sc:name ?name.

    ?poet wdt:P19 ?city .
    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}

"""

print("Results")
x=run_query(queryString)

Results
[('born', '4')]
1


In [30]:
# Given a city retrieve the Italian poets that were educated there
queryString = """
SELECT DISTINCT ?poet ?name
WHERE {

    ?poet wdt:P106 wd:Q49757 .        #BGP for poet
    ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
    
    ?poet wdt:P69 ?uni .              #BGP for educated at
    ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
    ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
    ?uni wdt:P131 ?city.              #BGP to retieve the city 

    ?poet sc:name ?name.

    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}

"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q5592'), ('name', 'Michelangelo')]
[('poet', 'http://www.wikidata.org/entity/Q560541'), ('name', 'Tommaso Landolfi')]
2


In [33]:
# Given a city retrieve the number Italian poets that were educated there
queryString = """
SELECT COUNT(DISTINCT (?poet)) as ?educated
WHERE {

    ?poet wdt:P106 wd:Q49757 .        #BGP for poet
    ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
    
    ?poet wdt:P69 ?uni .              #BGP for educated at
    ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
    ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
    ?uni wdt:P131 ?city.              #BGP to retieve the city 

    ?poet sc:name ?name.

    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}

"""

print("Results")
x=run_query(queryString)

Results
[('educated', '2')]
1


In [36]:
# Given a city retrieve the Italian poets that were born or educated there
queryString = """
SELECT ?city ?born ?educated
WHERE {

    {
        SELECT ?city COUNT(DISTINCT (?poet)) as ?born
        WHERE {
            ?poet wdt:P106 wd:Q49757 . #BGP for poet
            ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
            ?poet sc:name ?name.

            ?poet wdt:P19 ?city .
        }
    }
    UNION 
    {
        SELECT ?city COUNT(DISTINCT (?poet)) as ?educated
        WHERE {
            ?poet wdt:P106 wd:Q49757 .        #BGP for poet
            ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'

            ?poet wdt:P69 ?uni .              #BGP for educated at
            ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
            ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
            ?uni wdt:P131 ?city.              #BGP to retieve the city 
        }
    }

    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q2044'), ('born', '4')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('educated', '2')]
2


In [6]:
# Given a city retrieve the Italian poets that were born or educated there (removed BGP to impose that the university is in Italy)
queryString = """
SELECT ?city ?born ?educated
WHERE {

    {
        SELECT ?city COUNT(DISTINCT (?poet)) as ?born
        WHERE {
            ?poet wdt:P106 wd:Q49757 . #BGP for poet
            ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
            ?poet sc:name ?name.

            ?poet wdt:P19 ?city .
        }
    }
    UNION 
    {
        SELECT ?city COUNT(DISTINCT (?poet)) as ?educated
        WHERE {
            ?poet wdt:P106 wd:Q49757 .        #BGP for poet
            ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'

            ?poet wdt:P69 ?uni .              #BGP for educated at
            ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
            ?uni wdt:P131 ?city.              #BGP to retieve the city 
        }
    }

    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q2044'), ('born', '4')]
[('city', 'http://www.wikidata.org/entity/Q2044'), ('educated', '2')]
2


In [40]:
# Given a city retrieve the Italian poets that were born or educated there and sums the two counts
queryString = """
SELECT ?city SUM(?count) as ?count
WHERE {

    {
        #Number of poets born in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 . #BGP for poet
            ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
            ?poet sc:name ?name.

            ?poet wdt:P19 ?city .
        }
    }
    UNION 
    {
        #Number of poets that studied in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 .        #BGP for poet
            ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'

            ?poet wdt:P69 ?uni .              #BGP for educated at
            ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
            ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
            ?uni wdt:P131 ?city.              #BGP to retieve the city 
        }
    }

    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}
GROUP BY ?city

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q2044'), ('count', '6')]
1


In [7]:
# Given a city retrieve the Italian poets that were born or educated there and sums the two counts (removed BGP to impose that the university is in Italy)
queryString = """
SELECT ?city SUM(?count) as ?count
WHERE {

    {
        #Number of poets born in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 . #BGP for poet
            ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
            ?poet sc:name ?name.

            ?poet wdt:P19 ?city .
        }
    }
    UNION 
    {
        #Number of poets that studied in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 .        #BGP for poet
            ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'

            ?poet wdt:P69 ?uni .              #BGP for educated at
            ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
            ?uni wdt:P131 ?city.              #BGP to retieve the city 
        }
    }

    FILTER(?city = wd:Q2044)    #Filter only for city 'Florence'
}
GROUP BY ?city

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q2044'), ('count', '6')]
1


In [41]:
# Given a city retrieve the Italian poets were born or educated there and sums the two counts
queryString = """
SELECT ?city ?cityName SUM(?count) as ?count
WHERE {

    {
        #Get all the cities in which Italian potes were born or studied
        SELECT DISTINCT ?city ?cityName
        WHERE {
        
            {
                # Get all the cities in which Italian poets were born
                SELECT ?city ?cityName
                WHERE {
                   ?poet wdt:P106 wd:Q49757 .
                   ?poet wdt:P172 wd:Q50001 .
                   ?poet wdt:P19 ?city .
                   ?city sc:name ?cityName.
                }
            } UNION {
                # Get all the cities in which Italian poets attended university
                SELECT ?city ?cityName
                WHERE {
                   ?poet wdt:P106 wd:Q49757 .        #BGP for poet
                   ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
                   ?poet wdt:P69 ?uni .              #BGP for educated at
                   ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
                   ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
                   ?uni wdt:P131 ?city.              #BGP to retieve the city 
                   ?city sc:name ?cityName.
                }
            }
            
        }
    }

    {
        #Number of poets born in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 . #BGP for poet
            ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
            ?poet sc:name ?name.

            ?poet wdt:P19 ?city .
        }
    }
    UNION 
    {
        #Number of poets that studied in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 .        #BGP for poet
            ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'

            ?poet wdt:P69 ?uni .              #BGP for educated at
            ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
            ?uni wdt:P17 wd:Q38.              #BGP for country 'Italy'
            ?uni wdt:P131 ?city.              #BGP to retieve the city 
        }
    }

}
GROUP BY ?city ?cityName

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q2634'), ('cityName', 'Naples'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q220'), ('cityName', 'Rome'), ('count', '5')]
[('city', 'http://www.wikidata.org/entity/Q1449'), ('cityName', 'Genoa'), ('count', '4')]
[('city', 'http://www.wikidata.org/entity/Q219330'), ('cityName', 'Pietrasanta'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q279'), ('cityName', 'Modena'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q52069'), ('cityName', 'Caprese Michelangelo'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q83362'), ('cityName', 'Recanati'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q270613'), ('cityName', 'Celle Ligure'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q6247'), ('cityName', 'Mantua'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q20103'), ('cityName', 'Assisi'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q13375'), (

In [4]:
# Get all the poets that were born and attended university in the same city
queryString = """
SELECT ?pob ?cityName ?poet
WHERE {

       ?poet wdt:P106 wd:Q49757 .  # BGP for poet
       ?poet wdt:P172 wd:Q50001 .  # BGP for Italians
       ?poet wdt:P19 ?pob .        # BGP for place of birth
       
       ?poet wdt:P69 ?uni .              #BGP for educated at
       ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
       ?uni wdt:P131 ?uniCity.           #BGP to retieve the city 
       
       FILTER(?pob = ?uniCity)
       
       ?pob sc:name ?cityName.

}
"""

print("Results")
x=run_query(queryString)

Results
[('pob', 'http://www.wikidata.org/entity/Q1449'), ('cityName', 'Genoa'), ('poet', 'http://www.wikidata.org/entity/Q350103')]
[('pob', 'http://www.wikidata.org/entity/Q1449'), ('cityName', 'Genoa'), ('poet', 'http://www.wikidata.org/entity/Q350103')]
[('pob', 'http://www.wikidata.org/entity/Q220'), ('cityName', 'Rome'), ('poet', 'http://www.wikidata.org/entity/Q334147')]
[('pob', 'http://www.wikidata.org/entity/Q220'), ('cityName', 'Rome'), ('poet', 'http://www.wikidata.org/entity/Q334147')]
4


In [7]:
# Get all the poets that were born and attended university in the same city
queryString = """
SELECT ?pob ?cityName (COUNT(DISTINCT(?poet)) as ?count)
WHERE {

   ?poet wdt:P106 wd:Q49757 .  # BGP for poet
   ?poet wdt:P172 wd:Q50001 .  # BGP for Italians
   ?poet wdt:P19 ?pob .        # BGP for place of birth

   ?poet wdt:P69 ?uni .              #BGP for educated at
   ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
   ?uni wdt:P131 ?uniCity.           #BGP to retieve the city 

   FILTER(?pob = ?uniCity)

   ?pob sc:name ?cityName.

}
GROUP BY ?pob ?cityName
"""

print("Results")
x=run_query(queryString)

Results
[('pob', 'http://www.wikidata.org/entity/Q1449'), ('cityName', 'Genoa'), ('count', '1')]
[('pob', 'http://www.wikidata.org/entity/Q220'), ('cityName', 'Rome'), ('count', '1')]
2


Final query for this task

I assume italian potes are the one with `ethnic group` Italians, and I consider as university all intaces of `Q3918` or instances of a subclass.

In [9]:
# Given a city retrieve the Italian poets that were born or educated there and sums the two counts (removed BGP to impose that the university is in Italy)
queryString = """
SELECT ?city ?cityName SUM(?count) as ?count
WHERE {

    {
        #Get all the cities in which Italian potes were born or studied
        SELECT DISTINCT ?city ?cityName
        WHERE {
        
            {
                # Get all the cities in which Italian poets were born
                SELECT ?city ?cityName
                WHERE {
                   ?poet wdt:P106 wd:Q49757 .
                   ?poet wdt:P172 wd:Q50001 .
                   ?poet wdt:P19 ?city .
                   ?city sc:name ?cityName.
                }
            } UNION {
                # Get all the cities in which Italian poets attended university
                SELECT ?city ?cityName
                WHERE {
                   ?poet wdt:P106 wd:Q49757 .        #BGP for poet
                   ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'
                   ?poet wdt:P69 ?uni .              #BGP for educated at
                   ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
                   ?uni wdt:P131 ?city.              #BGP to retieve the city 
                   ?city sc:name ?cityName.
                }
            }
            
        }
    }

    {
        #Number of poets born in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 . #BGP for poet
            ?poet wdt:P172 wd:Q50001 . #BGP for 'Italians'
            ?poet sc:name ?name.

            ?poet wdt:P19 ?city .
        }
    }
    UNION 
    {
        #Number of poets that studied in the city
        SELECT ?city COUNT(DISTINCT (?poet)) as ?count
        WHERE {
            ?poet wdt:P106 wd:Q49757 .        #BGP for poet
            ?poet wdt:P172 wd:Q50001 .        #BGP for 'Italians'

            ?poet wdt:P69 ?uni .              #BGP for educated at
            ?uni wdt:P31/wdt:P279* wd:Q3918.  #BGP for universities
            ?uni wdt:P131 ?city.              #BGP to retieve the city 
        }
    }

}
GROUP BY ?city ?cityName

"""

print("Results")
x=run_query(queryString)

Results
[('city', 'http://www.wikidata.org/entity/Q1449'), ('cityName', 'Genoa'), ('count', '4')]
[('city', 'http://www.wikidata.org/entity/Q219330'), ('cityName', 'Pietrasanta'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q220'), ('cityName', 'Rome'), ('count', '5')]
[('city', 'http://www.wikidata.org/entity/Q2634'), ('cityName', 'Naples'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q6441'), ('cityName', 'Montpellier'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q279'), ('cityName', 'Modena'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q52069'), ('cityName', 'Caprese Michelangelo'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q83362'), ('cityName', 'Recanati'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q6247'), ('cityName', 'Mantua'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q20103'), ('cityName', 'Assisi'), ('count', '1')]
[('city', 'http://www.wikidata.org/entity/Q270613'), ('c

## Task 8
Return the italian poets who won the Nobel Prize in Literature (the result set must be poet IRI and label).

In [3]:
# Get info about significant people
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    ?person wdt:P3342 ?o .
    ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q15461487'), ('name', 'Marion Foale')]
[('o', 'http://www.wikidata.org/entity/Q15734554'), ('name', 'Thora Wigardh')]
[('o', 'http://www.wikidata.org/entity/Q213675'), ('name', 'Gottfried August Bürger')]
[('o', 'http://www.wikidata.org/entity/Q4972692'), ('name', 'Anna Petrus')]
[('o', 'http://www.wikidata.org/entity/Q4990759'), ('name', 'Ria Wägner')]
[('o', 'http://www.wikidata.org/entity/Q715083'), ('name', 'Joe Murphy')]
[('o', 'http://www.wikidata.org/entity/Q1450891'), ('name', 'Xavier Niel')]
[('o', 'http://www.wikidata.org/entity/Q16059906'), ('name', 'Vincenzo Petroccelli')]
[('o', 'http://www.wikidata.org/entity/Q47044502'), ('name', 'Ida Mitchell Eliot')]
[('o', 'http://www.wikidata.org/entity/Q98927523'), ('name', 'Marie Duclos Fretageot')]
10


In [4]:
# Get info about significant people that are poet and 'Italians' as 'ethnic group'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    ?person wdt:P3342 ?o .       #significant person
    ?person wdt:P106 wd:Q49757.  #occupation is 'poet'
    ?person wdt:P172 wd:Q50001 . #ethnic group is italian
    ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q18197540'), ('name', 'René Dalize')]
[('o', 'http://www.wikidata.org/entity/Q1401'), ('name', 'Petrarch')]
[('o', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio')]
[('o', 'http://www.wikidata.org/entity/Q100901055'), ('name', 'Carolina Cristofori Piva')]
4


In [4]:
# Get info about significant people that are poet and 'Italians' as 'ethnic group'
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
    ?person wdt:P3342 ?o .       #significant person
    ?person wdt:P106 wd:Q49757.  #occupation is 'poet'
    ?person wdt:P172 wd:Q50001 . #ethnic group is italian
    ?o sc:name ?name.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q18197540'), ('name', 'René Dalize')]
[('o', 'http://www.wikidata.org/entity/Q1401'), ('name', 'Petrarch')]
[('o', 'http://www.wikidata.org/entity/Q1402'), ('name', 'Giovanni Boccaccio')]
[('o', 'http://www.wikidata.org/entity/Q100901055'), ('name', 'Carolina Cristofori Piva')]
4


In [7]:
# Get info about significant people
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
    ?person wdt:P3342 ?o .
    ?person ?p ?o. 
    ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('name', 'significant person')]
[('p', 'http://www.wikidata.org/prop/direct/P488'), ('name', 'chairperson')]
[('p', 'http://www.wikidata.org/prop/direct/P737'), ('name', 'influenced by')]
[('p', 'http://www.wikidata.org/prop/direct/P802'), ('name', 'student')]
[('p', 'http://www.wikidata.org/prop/direct/P453'), ('name', 'character role')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P451'), ('name', 'unmarried part

In [14]:
# Get info about 'winning'
queryString = """
SELECT DISTINCT ?sname ?oname
WHERE {
    ?o wdt:P1346 ?s. 
    ?s sc:name ?sname.
    ?o sc:name ?oname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Taliana Vargas'), ('oname', 'Miss Colombia 2007')]
[('sname', 'Illya Kvasha'), ('oname', "2013 European Diving Championships – men's 1 metre springboard")]
[('sname', 'Egoitz García'), ('oname', '2014 Tropicale Amissa Bongo')]
[('sname', 'Guillaume Van Keirsbulck'), ('oname', '2014 Tour of Qatar')]
[('sname', 'Jason Day'), ('oname', '2014 WGC-Accenture Match Play Championship')]
[('sname', 'Bentonville High School'), ('oname', '2014 Arkansas High School 7A Boys Soccer Season')]
[('sname', 'UE Engordany'), ('oname', '2013–14 Segona Divisió')]
[('sname', 'Bert Pronk'), ('oname', '1977 Tour de Suisse')]
[('sname', 'Pierre Barthès'), ('oname', "1970 US Open – Men's Doubles")]
[('sname', 'BC Zepter Vienna'), ('oname', '2012–13 Österreichische Basketball Bundesliga')]
[('sname', 'Dylan Girdlestone'), ('oname', 'Tour du Rwanda 2013')]
[('sname', 'Sven Van Luijk'), ('oname', '2012 Czech Cycling Tour')]
[('sname', 'Kristian Haugaard'), ('oname', '2013 Czech Cycling Tour')]
[

In [16]:
# Get info about 'winning'
queryString = """
SELECT DISTINCT ?sname ?oname
WHERE {
    ?o wdt:P1346 ?s. 
    ?s sc:name ?sname.
    ?o sc:name ?oname.
    FILTER REGEX(?o, "[Nn]obel")
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [19]:
# Get info about prizes won by potes
queryString = """
SELECT DISTINCT ?sname ?prize
WHERE {
    ?poet wdt:P106 wd:Q49757 .
    ?prize wdt:P1346 ?poet. 
    ?prize sc:name ?sname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Mercury Prize'), ('prize', 'http://www.wikidata.org/entity/Q501876')]
[('sname', 'Jakob Wassermann Prize'), ('prize', 'http://www.wikidata.org/entity/Q1678870')]
[('sname', 'Filmfare Award for Best Story'), ('prize', 'http://www.wikidata.org/entity/Q1414504')]
[('sname', 'Filmfare Award for Best Dialogue'), ('prize', 'http://www.wikidata.org/entity/Q1414514')]
[('sname', 'Ruth Lilly Poetry Prize'), ('prize', 'http://www.wikidata.org/entity/Q728657')]
[('sname', 'Dos Passos Prize'), ('prize', 'http://www.wikidata.org/entity/Q2439701')]
[('sname', 'Nobel Prize in Literature'), ('prize', 'http://www.wikidata.org/entity/Q37922')]
[('sname', 'Mercè Rodoreda Award'), ('prize', 'http://www.wikidata.org/entity/Q11700304')]
[('sname', 'Grammy Award for Best Spoken Word Album'), ('prize', 'http://www.wikidata.org/entity/Q5593890')]
[('sname', 'De Gyldne Laurbær'), ('prize', 'http://www.wikidata.org/entity/Q1179930')]
10


In [19]:
# Get info about prizes won by potes
queryString = """
SELECT DISTINCT ?sname ?prize
WHERE {
    ?poet wdt:P106 wd:Q49757 .
    ?prize wdt:P1346 ?poet. 
    ?prize sc:name ?sname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('sname', 'Mercury Prize'), ('prize', 'http://www.wikidata.org/entity/Q501876')]
[('sname', 'Jakob Wassermann Prize'), ('prize', 'http://www.wikidata.org/entity/Q1678870')]
[('sname', 'Filmfare Award for Best Story'), ('prize', 'http://www.wikidata.org/entity/Q1414504')]
[('sname', 'Filmfare Award for Best Dialogue'), ('prize', 'http://www.wikidata.org/entity/Q1414514')]
[('sname', 'Ruth Lilly Poetry Prize'), ('prize', 'http://www.wikidata.org/entity/Q728657')]
[('sname', 'Dos Passos Prize'), ('prize', 'http://www.wikidata.org/entity/Q2439701')]
[('sname', 'Nobel Prize in Literature'), ('prize', 'http://www.wikidata.org/entity/Q37922')]
[('sname', 'Mercè Rodoreda Award'), ('prize', 'http://www.wikidata.org/entity/Q11700304')]
[('sname', 'Grammy Award for Best Spoken Word Album'), ('prize', 'http://www.wikidata.org/entity/Q5593890')]
[('sname', 'De Gyldne Laurbær'), ('prize', 'http://www.wikidata.org/entity/Q1179930')]
10


In [3]:
# Get all italian potes
queryString = """
SELECT DISTINCT ?person ?name
WHERE {
    ?person wdt:P106 wd:Q49757.    #occupation is 'poet'
    ?person wdt:P172 wd:Q50001 .   #ethnic group is italian
    ?person sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q296272'), ('name', 'Pietro Aretino')]
[('person', 'http://www.wikidata.org/entity/Q982512'), ('name', 'Giuseppe Giusti')]
[('person', 'http://www.wikidata.org/entity/Q55433'), ('name', 'Michelangelo Antonioni')]
[('person', 'http://www.wikidata.org/entity/Q83321'), ('name', 'Giacomo Casanova')]
[('person', 'http://www.wikidata.org/entity/Q182046'), ('name', 'Leon Battista Alberti')]
[('person', 'http://www.wikidata.org/entity/Q48900'), ('name', 'Ludovico Ariosto')]
[('person', 'http://www.wikidata.org/entity/Q165534'), ('name', 'Carlo Goldoni')]
[('person', 'http://www.wikidata.org/entity/Q7803'), ('name', 'Bronzino')]
[('person', 'http://www.wikidata.org/entity/Q982387'), ('name', 'Giovanni Capurro')]
[('person', 'http://www.wikidata.org/entity/Q1067'), ('name', 'Dante Alighieri')]
[('person', 'http://www.wikidata.org/entity/Q346250'), ('name', 'Ada Negri')]
[('person', 'http://www.wikidata.org/entity/Q166234'), ('name', 'Ugo Foscol

In [4]:
# Get all italian potes that won prizes with their prizes
queryString = """
SELECT DISTINCT ?person ?name ?prize ?pname
WHERE {
    ?person wdt:P106 wd:Q49757.    #occupation is 'poet'
    ?person wdt:P172 wd:Q50001 .   #ethnic group is italian
    ?person sc:name ?name.
    
    ?person wdt:P166 ?prize.
    ?prize sc:name ?pname.
    
}
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q55433'), ('name', 'Michelangelo Antonioni'), ('prize', 'http://www.wikidata.org/entity/Q7650000'), ('pname', 'Sutherland Trophy')]
[('person', 'http://www.wikidata.org/entity/Q53009'), ('name', 'Bernardo Bertolucci'), ('prize', 'http://www.wikidata.org/entity/Q7650000'), ('pname', 'Sutherland Trophy')]
[('person', 'http://www.wikidata.org/entity/Q216582'), ('name', 'Filippo Tommaso Marinettiz6eztfzitryxi'), ('prize', 'http://www.wikidata.org/entity/Q1541614'), ('pname', 'War Merit Cross (Italy)')]
[('person', 'http://www.wikidata.org/entity/Q560541'), ('name', 'Tommaso Landolfi'), ('prize', 'http://www.wikidata.org/entity/Q1526935'), ('pname', 'Viareggio Prize')]
[('person', 'http://www.wikidata.org/entity/Q560541'), ('name', 'Tommaso Landolfi'), ('prize', 'http://www.wikidata.org/entity/Q788568'), ('pname', 'Bagutta Prize')]
[('person', 'http://www.wikidata.org/entity/Q53009'), ('name', 'Bernardo Bertolucci'), ('prize', 'http://www.

Final query for this task

In [6]:
# Get all italian potes that won nobel prize in literature
queryString = """
SELECT DISTINCT ?person ?name ?prize ?pname
WHERE {
    ?person wdt:P106 wd:Q49757.    #occupation is 'poet'
    ?person wdt:P172 wd:Q50001 .   #ethnic group is italian
    ?person sc:name ?name.
    
    ?person wdt:P166 wd:Q37922.    #BGP to specify person won a nobel prize in literature
}
"""

print("Results")
x=run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q43440'), ('name', 'Giosuè Carducci')]
[('person', 'http://www.wikidata.org/entity/Q1403'), ('name', 'Luigi Pirandello')]
[('person', 'http://www.wikidata.org/entity/Q765'), ('name', 'Dario Fo')]
3
